# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [1]:
pip install category_encoders

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install scikit-learn==1.1.3

In [3]:
#Подгружаем все библиотеки в одной ячейке, список может пополняться :)
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn 
import matplotlib.pyplot as plt
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce
from category_encoders import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [4]:
try:
    data = pd.read_csv('/datasets/autos.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/autos.csv')

In [5]:
data

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,2016-03-21 09:50:58,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,2016-03-21 00:00:00,0,2694,2016-03-21 10:42:49
354365,2016-03-14 17:48:27,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
354366,2016-03-05 19:56:21,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
354367,2016-03-19 18:57:12,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26


In [6]:
data.info()
print(80*'=')
display(data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [7]:
data.head(10)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07
6,2016-04-01 20:48:51,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,2016-04-01 00:00:00,0,67112,2016-04-05 18:18:39
7,2016-03-21 18:54:38,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,2016-03-21 00:00:00,0,19348,2016-03-25 16:47:58
8,2016-04-04 23:42:13,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,2016-04-04 00:00:00,0,94505,2016-04-04 23:42:13
9,2016-03-17 10:53:50,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,2016-03-17 00:00:00,0,27472,2016-03-31 17:17:06


Заметил что столбец 0, 12, 15 неправильного типа. Им присвоено object, вместо полагаемого DateTime

In [8]:
#Исправили замечание
data['DateCrawled'] = pd.to_datetime(data['DateCrawled'], format='%Y-%m-%d %H:%M:%S')
data['DateCreated'] = pd.to_datetime(data['DateCreated'], format='%Y-%m-%d %H:%M:%S')
data['LastSeen'] = pd.to_datetime(data['LastSeen'], format='%Y-%m-%d %H:%M:%S')

In [9]:
data = data.drop_duplicates(keep='last')
data.duplicated().sum()

0

Обнаружено и удалено 4 явных дубликата

In [10]:
data.fillna(value={'VehicleType': 'unknown', 
                   'Gearbox': 'unknown',
                   'Model': 'unknown', 
                   'FuelType': 'unknown', 
                   'NotRepaired': 'unknown'}, inplace=True)
# Заполнение пропущенных значений

In [11]:
data = data[data['Price'] > 100]
data = data[data['Power'] < 1000]
#Удалили аномальные значения

In [12]:
data = data.loc[(data['RegistrationYear'] <= 2021) & (data['RegistrationYear'] >= 1900)]

<b>В ходе этапа обработки данных сделали следующее:<br>
1. Исправили неправильные типы данных
2. Удалили явные дубликаты
3. Заполнили пропущенные значения 
4. Удалили аномальные значения

<b> Допустим, можно было бы обработать FuelType, заполнив значениями по такой-же модели автомобиля<br>
Достать категории GearBox тоже наверняка как-то можно, но вроде это не целесообразно, данных должно быть достаточно для прогнозирования.<br>
    Модель восстановить даже по бренду не является возможным, как и узнать NotRepaired. Удалять данные не разумно ибо от датасета бублик останется)


 Самое интересное, что на нескольких сайтов по продажам подержанных автомобилей в Германии (страну можно определить по почтовым индексам, большинство из них из Германии) показало, что цены начинаются действительно с 0 и 1 евро, но это единичные объявления и не понятно, то ли это ошибка при заполнении формы, то ли машины на металлолом. Но я  все-таки склоняюсь к тому, чтобы избавиться от таких данных



## Обучение моделей

In [13]:
# Разделение на признаки и целевую переменную
features = data.drop('Price', axis=1)
target = data['Price']

# Разделение на обучающую и тестовую выборки
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=42)
features_train, features_val, target_train, target_val = train_test_split(features_train, target_train, test_size=0.2, random_state=42)

# Преобразование столбца с типом данных datetime64 в числовой формат
features_train['DateCrawled'] = features_train['DateCrawled'].astype('int64')
features_test['DateCrawled'] = features_test['DateCrawled'].astype('int64')
features_val['DateCrawled'] = features_val['DateCrawled'].astype('int64')

# Создание наборов данных для разных методов кодировки
# Набор данных 1: TargetEncoder и BinaryEncoder для всех моделей
encoder1 = TargetEncoder(cols=['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired'])
features_train_enc1 = encoder1.fit_transform(features_train, target_train)
features_test_enc1 = encoder1.transform(features_test)
features_val_enc1 = encoder1.transform(features_val)

# Удаление дат из набора данных
features_train_enc1.drop(['DateCrawled', 'DateCreated', 'LastSeen'], axis=1, inplace=True)
features_test_enc1.drop(['DateCrawled', 'DateCreated', 'LastSeen'], axis=1, inplace=True)
features_val_enc1.drop(['DateCrawled', 'DateCreated', 'LastSeen'], axis=1, inplace=True)

# Набор данных 2: OneHotEncoder для линейных моделей и TargetEncoder для остальных моделей
encoder2 = OneHotEncoder(handle_unknown='ignore', drop='first')
encoder2.fit(features_train[['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']])
features_train_enc2 = encoder2.transform(features_train[['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']])
features_test_enc2 = encoder2.transform(features_test[['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']])
features_val_enc2 = encoder2.transform(features_val[['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']])

# Объединение закодированных категориальных признаков с числовыми признаками
features_train_enc2 = hstack((features_train_enc2, features_train[['RegistrationYear', 'RegistrationMonth', 'Kilometer', 'PostalCode', 'DateCrawled']]))
features_test_enc2 = hstack((features_test_enc2, features_test[['RegistrationYear', 'RegistrationMonth', 'Kilometer', 'PostalCode', 'DateCrawled']]))
features_val_enc2 = hstack((features_val_enc2, features_val[['RegistrationYear', 'RegistrationMonth', 'Kilometer', 'PostalCode', 'DateCrawled']]))

# Преобразование разреженной матрицы в формат CSC и удаление ненужных столбцов
features_train_enc2 = features_train_enc2.tocsc()[:, :-4]
features_test_enc2 = features_test_enc2.tocsc()[:, :-4]
features_val_enc2 = features_val_enc2.tocsc()[:, :-4]

In [14]:
%%time
# Линейная регрессия с OneHotEncoder
linear_model = LinearRegression()
linear_model.fit(features_train_enc2, target_train)


CPU times: user 14.9 s, sys: 19 s, total: 33.9 s
Wall time: 33.9 s


LinearRegression()

In [15]:
%%time
linear_predictions = linear_model.predict(features_train_enc2)
linear_rmse = np.sqrt(mean_squared_error(target_train, linear_predictions))

CPU times: user 18.7 ms, sys: 10.8 ms, total: 29.5 ms
Wall time: 87.5 ms


In [16]:
%%time
linear_predictions_val = linear_model.predict(features_val_enc2)
linear_rmse_val = mean_squared_error(target_val, linear_predictions_val, squared=False)

CPU times: user 2.42 ms, sys: 0 ns, total: 2.42 ms
Wall time: 2.04 ms


In [17]:
print('RMSE на валидационной выборке:', linear_rmse_val,
      '\nRMSE на тренировочной выборке:', linear_rmse)

RMSE на валидационной выборке: 3093.921980795153 
RMSE на тренировочной выборке: 3086.429889183905



Нам нужно время обучения и время предсказания  вычислить  и проанализировать отдельно друг от друга
    
- время обучения это  чистый `.fit()` модели - без подбора гиперпараметров и без предсказаний,  то есть время обучения gridsearch не подойдет
    
- время предсказания это только `.predict()` без обучения



In [18]:
%%time
# Случайный лес с TargetEncoder и BinaryEncoder
rf_model = RandomForestRegressor()
rf_model.fit(features_train_enc1, target_train)

CPU times: user 1min 42s, sys: 1.46 s, total: 1min 44s
Wall time: 1min 44s


RandomForestRegressor()

In [19]:
%%time
rf_predictions = rf_model.predict(features_train_enc1)
rf_rmse = np.sqrt(mean_squared_error(target_train, rf_predictions))

CPU times: user 11.6 s, sys: 59.2 ms, total: 11.6 s
Wall time: 11.6 s


In [20]:
%%time
rf_predictions_val = rf_model.predict(features_val_enc1)
rf_rmse_val = mean_squared_error(target_val, rf_predictions_val, squared=False)

CPU times: user 2.99 s, sys: 35.4 ms, total: 3.02 s
Wall time: 3.03 s


In [21]:
print('RMSE на валидационной выборке:', rf_rmse_val,
      '\nRMSE на тренировочной выборке', rf_rmse)

RMSE на валидационной выборке: 1570.5504527746757 
RMSE на тренировочной выборке 595.4886696851785


In [22]:
%%time
lgbm_model = LGBMRegressor()
lgbm_model.fit(features_train_enc1, target_train)

CPU times: user 3min 59s, sys: 2.13 s, total: 4min 1s
Wall time: 4min 3s


LGBMRegressor()

In [23]:
%%time
lgbm_predictions = lgbm_model.predict(features_train_enc1)
lgbm_rmse = np.sqrt(mean_squared_error(target_train, lgbm_predictions))

CPU times: user 1.78 s, sys: 21.5 ms, total: 1.8 s
Wall time: 1.8 s


In [24]:
%%time
lgbm_predictions_val = lgbm_model.predict(features_val_enc1)
lgbm_rmse_val = np.sqrt(mean_squared_error(target_val, lgbm_predictions_val))

CPU times: user 501 ms, sys: 0 ns, total: 501 ms
Wall time: 505 ms


In [25]:
print('RMSE на валидационной выборке:', lgbm_rmse_val,
      '\nRMSE на тренировочной выборке', lgbm_rmse)

RMSE на валидационной выборке: 1713.8247527045348 
RMSE на тренировочной выборке 1683.5564367649731


<b> Вывод по разделу: <br>
    <br>Проделали огромную работу от разделения на выборки и кодирование данных до обучения и вычисления метрик. Экспериментальным методом получили:<br>
   <br> 1. Не конкурентно способность линейной регрессии, значение RMSE превысило 2500, что является плохим показателем, но она имеет лучшее время обучения.<br>
    <br>2. Отличный результат показало Случайное дерево решений, значение метрики RMSE достигает порядка 1700, что является лучшим результатом. Так-же время обучения менее 2-ух минут, что является отличным временем обучения в нашей ситуации.<br>
    <br>3. Результаты которая показала модель LGMB ухудшились после тестирования на валидационной выборке. Значение метрики получили почти 1800, что является не лучшим и не худшим показателем между двумя предыдущими моделями, а так-же время обучения целых 4 минуты, что является худшим показателем. <br>
    <br>Учитывая все полученные данные, будет оптимальным выбрать модель Случайный лес, т.к. значение метрики RMSE не превысило 2500 и мы имеем отличные результаты в скорости обучения, дальнейший анализ и тестирование будем проводить на ней.

## Анализ моделей

In [30]:
%%time
best_model = rf_model

best_predictions = best_model.predict(features_test_enc1)
best_rmse = np.sqrt(mean_squared_error(target_test, best_predictions))
best_rmse_val = rf_rmse_val

print('RMSE на валидационной выборке:', rf_rmse_val,
      '\nRMSE на тестовой выборке', best_rmse)

RMSE на валидационной выборке: 1570.5504527746757 
RMSE на тестовой выборке 1584.7116601769037
CPU times: user 3.74 s, sys: 398 µs, total: 3.74 s
Wall time: 3.74 s


<b>Значение RMSE на тестовой выборке всего 1584. Это является хорошим результатом, мы имеем право оставить эту модель как лучшую из представленных.

<b> Вывод: <br>
Нам важно: <br>
    <br>1. качество предсказаний
    <br>2. скорость предсказаний 
    <br>3. время обучения

Под критерии подходит модель Cлучайного леса, она дала нам хорошее качество предсказания, а так-же лучшее время обучения для представленного качества. Качество предсказаний полностью удовлетворяет, RMSE не достигает даже 1600, поэтому модель имеет право на дальнейшую жизнь. Так-как LGBM показала плохую скорость обучения, то мы не можем ее использовать хотя она и дала неплохое качество. Линейная модель быстро обучается, но медленно предсказывает и делает это крайне плохо. 